In [1]:

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
from dqn import DQNAgent
from environment import environment, candle_class
from transformer_layer import TransformerBlock, PositionEmbedding

import time
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd    
import numpy as np
from collections import deque
from tqdm import tqdm
import random
import cv2
import pickle

name = "dqn_trading_transformer"
data_dir = "./archive"
resume = True
#resume = False

warmup_parallel = 6
train_parallel = 6
warmup_steps = 1000

lr = 0.0001
memory_size = 15000
gamma = 0.9975
exploration = 0.01
target_model_sync = 100
batch_size = 16

dlen = 120
pos_size = 0.05 * 100000
comm = 8/100000
res_high = 100

x = environment(data_dir, dlen, res_high, comm, pos_size)
m5,m15,h1,h4,d1,pos = x.reset(True)

for _ in range(10):
    obs, _,_ = x.step(0)
    m5,m15,h1,h4,d1,pos = obs
    m5 +=1 
    m5/=2
    m15 +=1 
    m15/=2
    h1 +=1 
    h1/=2
    h4 +=1 
    h4/=2
    d1 +=1 
    d1/=2
    cv2.imshow("m5",cv2.resize(m5,(300,300)))
    cv2.imshow("m15",cv2.resize(m15,(300,300)))
    cv2.imshow("h1",cv2.resize(h1,(300,300)))
    cv2.imshow("h4",cv2.resize(h4,(300,300)))
    cv2.imshow("d1",cv2.resize(d1,(300,300)))
    cv2.waitKey(10)

plt.plot(m5[-1])
plt.show()
plt.imshow(m5[0:-1])
plt.show()

In [2]:
def make_model():
    
    D5 = tf.keras.layers.Dense(512)
    D6 = tf.keras.layers.Dense(256)
  
    LR = tf.keras.layers.LeakyReLU()

    LN1 = tf.keras.layers.LayerNormalization()
    num_tx_units = res_high
    P1 =  PositionEmbedding(dlen+1, num_tx_units)
    T1 = TransformerBlock(num_tx_units, 12, 256)
    T2= TransformerBlock(num_tx_units, 12, 256)
    T3 = TransformerBlock(num_tx_units, 12, 256)
    T4 = TransformerBlock(num_tx_units, 12, 256)
    T5 = TransformerBlock(num_tx_units, 12, 256)
    T6 = TransformerBlock(num_tx_units, 12, 256)
    T7 = TransformerBlock(num_tx_units, 12, 256)
    T8 = TransformerBlock(num_tx_units, 12, 256)
    T9 = TransformerBlock(num_tx_units, 12, 256)
    T10 = TransformerBlock(num_tx_units, 12, 256)

    cn = tf.keras.layers.Concatenate()

    GR1 = tf.keras.layers.GRU(512)
    def proc_chart(x):
        #print(x.shape)
        
        times = x[::,-1]
        x1 = x[::,:-1]
        x1 = tf.keras.layers.Reshape((res_high, dlen+1, 1))(x1)
        #print(times.shape)
        #print(x1.shape)

        x1 = tf.transpose(x1,perm=[0, 2, 1, 3])
        x1 = tf.keras.layers.Reshape((dlen+1, res_high))(x1)
        #print(x1.shape, dlen, res_high)

        x1 = P1(x1)
        x2 = tf.keras.layers.Embedding(input_dim=int(24*60/5), output_dim=res_high, input_length = dlen)(times)
        x1=x2+x1
        x1 = T1(x1)
        x1 = T2(x1)
        x1 = T3(x1)
        x1 = T4(x1)
        x1 = T5(x1)
        x1 = T6(x1)
        x1 = T7(x1)
        x1 = T8(x1)
        x1 = T9(x1)
        x1 = T10(x1)
        

        x1 = D5(x1)
        x1 = LR(x1)
        x1 = D6(x1)
        x1 = LR(x1)
        #x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
        x1 = GR1(x1)

        x1 = tf.keras.layers.Dense(1024,activity_regularizer=tf.keras.regularizers.L2(0.00001))(x1)
        x1 = LR(x1)
        x1 = tf.keras.layers.Dense(1024,activity_regularizer=tf.keras.regularizers.L2(0.00001))(x1)
        x1 = LR(x1)
        x1 = tf.keras.layers.Dense(1024,activity_regularizer=tf.keras.regularizers.L2(0.00001))(x1)
        x1 = LR(x1)
        x1 = tf.keras.layers.Dense(256,activity_regularizer=tf.keras.regularizers.L2(0.00001))(x1)
        x1 = LR(x1)
        #x1 = tf.keras.layers.LayerNormalization()(x1)
        return x1
    
    input_m5 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    input_m15 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    input_h1 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    input_h4 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    input_d1 = tf.keras.layers.Input(shape = (res_high+1, dlen+1))
    
    x0 = proc_chart(input_m5)
    x1 = proc_chart(input_m15)
    x2 = proc_chart(input_h1)
    x3 = proc_chart(input_h4)
    x4 = proc_chart(input_d1)
    
    input_net_position = tf.keras.layers.Input(shape = (1))


    x =cn([x0,x1,x2,x3,x4,input_net_position])
    
    x = tf.keras.layers.Dense(1024)(x)
    x = LR(x)
    x = tf.keras.layers.Dense(1024)(x)
    x = LR(x)
    x = tf.keras.layers.Dense(1024)(x)
    x = LR(x)
    
    outputs = tf.keras.layers.Dense(2, activation = "linear", use_bias=False, dtype="float32")(x)
    model = tf.keras.Model([input_m5,input_m15,input_h1,input_h4, input_d1, input_net_position], outputs)
    return model

In [3]:
tf.keras.backend.clear_session()

model = make_model()
target_model = make_model() #tf.keras.models.clone_model(model) does not work on shared layers
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 101, 121)]   0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 100, 121)     0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 100, 121, 1)  0           tf.__operators__.getitem_1[0][0] 
__________________________________________________________________________________________________
tf.compat.v1.transpose (TFOpLam (None, 121, 100, 1)  0           reshape[0][0]                    
______________________________________________________________________________________________

In [4]:
opt = tf.keras.optimizers.Adam(lr)




agent = DQNAgent(
    model = model, 
    target_model = target_model,
    n_actions = 2, 
    memory_size = memory_size, 
    gamma=gamma,
    optimizer = opt,
    batch_size = batch_size, 
    target_model_sync = target_model_sync,
    exploration = exploration,
    name=name,
    output_dir = "tx_8/")

#resume = False
if resume:
	print("loading weights...")
	agent.load_weights()

loading weights...


In [6]:
x = [environment(data_dir, dlen, res_high, comm, pos_size, False) for _ in range(warmup_parallel)]
print("warmup...")
n = warmup_steps
agent.train(num_steps = n, envs = x, warmup = n, log_interval = n)

warmup...
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
1000/1000 [==============================] - 183s 176ms/step - loss: 0.0000e+00 - mean q: 0.0000e+00 - rewards: -0.1311 - t: 175.8477


c:\users\root\appdata\local\programs\python\python36\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\root\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
len(agent.memory)

6000

In [ ]:
x = [environment(data_dir, dlen, res_high, comm, pos_size, False) for _ in range(train_parallel)]
print("training...")
n = 1000000000
agent.train(num_steps = n, envs = x, warmup = 0, log_interval = 1000)
print("done")

training...
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
using ./archive/NZDUSD.csv_m5.csv_candle_classes
   1/1000 [..............................] - ETA: 6:15 - loss: 0.0000e+00 - mean q: 0.0000e+00 - rewards: -0.5974 - t: 375.8237

In [ ]:
rew = [i[2] for i in agent.memory]
sorted(rew)[0:10], sorted(rew)[-10:][::-1], " - ", np.mean([abs(i) for i in rew])

In [ ]:
agent.save_weights()